# SQL Basics with DuckDB Sudan Extension

## Introduction

This notebook is a short introduction to SQL using the [DuckDB Sudan Extension](https://github.com/Osman-Geomatics93/duckdb-sudan-). It provides unified SQL access to Sudan's humanitarian, development, and geospatial data from 5 international APIs.

## Datasets

The following datasets are used in this notebook. No downloads needed — data is fetched live from international APIs or embedded in the extension.

- **SUDAN_States()** — 18 states with bilingual names (Arabic/English), ISO codes, centroids, and polygon boundaries
- **SUDAN_WorldBank()** — World Development Indicators (population, GDP, etc.)
- **SUDAN_Providers()** — List of 5 data providers
- **SUDAN_Boundaries()** — Administrative boundaries as GeoJSON (GADM v4.1)

## Supported Countries

| ISO3 | Country |
|:----:|---------|
| SDN | Sudan |
| EGY | Egypt |
| ETH | Ethiopia |
| TCD | Chad |
| SSD | South Sudan |
| ERI | Eritrea |
| LBY | Libya |
| CAF | Central African Republic |

## References

- [DuckDB SQL Introduction](https://duckdb.org/docs/sql/introduction.html)
- [W3Schools SQL Tutorial](https://www.w3schools.com/sql)
- [Sudan Extension Documentation](https://osman-geomatics93.github.io/duckdb-sudan-/)

## Installation

Run the following cell to install the required packages. The Sudan extension requires **DuckDB v1.4.4** — the version must match exactly.

In [ ]:
%pip install duckdb==1.4.4 duckdb-engine jupysql sqlalchemy --force-reinstall --no-cache-dir

## Library Import and Configuration

In [ ]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

Set configurations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

## Connecting to DuckDB and Loading the Sudan Extension

We connect to DuckDB with `allow_unsigned_extensions` enabled (required for custom extensions), then install and load the Sudan extension from the online repository.

In [ ]:
# Connect to DuckDB with unsigned extensions enabled
import duckdb

conn = duckdb.connect(config={'allow_unsigned_extensions': 'true'})

# Install and load the Sudan extension from the online repository
conn.execute("INSTALL httpfs; LOAD httpfs;")
conn.execute("SET custom_extension_repository = 'https://osman-geomatics93.github.io/duckdb-sudan-';")
conn.execute("INSTALL sudan; LOAD sudan;")

# Register the connection with jupysql using SQLAlchemy
from sqlalchemy import create_engine
engine = create_engine("duckdb:///:memory:", creator=lambda: conn.cursor())
%sql engine --alias duckdb

print("Sudan extension loaded successfully!")

Verify the extension is working by listing all data providers.

In [ ]:
%%sql

SELECT * FROM SUDAN_Providers();

If your SQL query is one line only, you may use the `%sql` magic command. For multi-line SQL queries, use the `%%sql` magic command.

## Explore Available Datasets

Let's look at the data the extension provides. Sudan has 18 states with bilingual names and polygon boundaries.

In [ ]:
%%sql

SELECT state_name, state_name_ar, iso_code, centroid_lon, centroid_lat
FROM SUDAN_States();

## Create Tables

Create a table named `states` from Sudan's 18 states.

In [ ]:
%%sql

CREATE TABLE states AS
SELECT state_name, state_name_ar, iso_code, centroid_lon, centroid_lat
FROM SUDAN_States();

Create a table named `population` from the World Bank API (Sudan + neighbors, recent years).

> **Note:** This cell fetches live data from the World Bank API. It may take a few seconds.

In [ ]:
%%sql

CREATE TABLE population AS
SELECT indicator_id, indicator_name, country, country_name, year, value
FROM SUDAN_WorldBank('SP.POP.TOTL', countries := ['SDN', 'EGY', 'ETH', 'TCD', 'SSD', 'ERI', 'LBY', 'CAF'])
WHERE value IS NOT NULL;

Create a table named `gdp` from World Bank GDP data.

> **Note:** GDP (current US$) indicator code is `NY.GDP.MKTP.CD`.

In [ ]:
%%sql

CREATE TABLE gdp AS
SELECT indicator_id, indicator_name, country, country_name, year, value
FROM SUDAN_WorldBank('NY.GDP.MKTP.CD', countries := ['SDN', 'EGY', 'ETH', 'TCD', 'SSD', 'ERI', 'LBY', 'CAF'])
WHERE value IS NOT NULL;

Display the table contents.

In [ ]:
%%sql

FROM states;

In [ ]:
%%sql

FROM population LIMIT 10;

In [ ]:
%%sql

FROM gdp LIMIT 10;

## The SQL SELECT Statement

The `SELECT` statement is used to select data from a database. Use either `SELECT *` to select all columns, or `SELECT column1, column2, ...` to select specific columns.

`SELECT * FROM states` is the same as `FROM states`.

In [ ]:
%%sql

SELECT * FROM states;

To limit the number of rows returned, use the `LIMIT` keyword. For example, `SELECT * FROM population LIMIT 10` will return only the first 10 rows.

In [ ]:
%%sql

SELECT * FROM population LIMIT 10;

Select a subset of columns from the `population` table and display the first 10 rows.

In [ ]:
%%sql

SELECT country_name, year, value FROM population LIMIT 10;

To select distinct values, use the `DISTINCT` keyword. For example, `SELECT DISTINCT country_name FROM population` returns only the unique country names.

In [ ]:
%%sql

SELECT DISTINCT country_name FROM population;

To count the number of rows returned, use the `COUNT(*)` function.

In [ ]:
%%sql

SELECT COUNT(*) FROM states;

In [ ]:
%%sql

SELECT COUNT(*) FROM population;

To count the number of distinct values, use the `COUNT(DISTINCT column)` function.

In [ ]:
%%sql

SELECT COUNT(DISTINCT country_name) FROM population;

To calculate the maximum value, use the `MAX(column)` function. For example, the maximum population recorded across all countries and years.

In [ ]:
%%sql

SELECT MAX(value) AS max_population FROM population;

To calculate the minimum value, use the `MIN(column)` function.

In [ ]:
%%sql

SELECT MIN(value) AS min_population FROM population;

To calculate the total value, use the `SUM(column)` function. For example, the total population of all 8 countries in 2023.

In [ ]:
%%sql

SELECT SUM(value) AS total_population
FROM population
WHERE year = 2023;

To calculate the average value, use the `AVG(column)` function.

In [ ]:
%%sql

SELECT ROUND(AVG(value), 0) AS avg_population
FROM population
WHERE year = 2023;

To order the results, use the `ORDER BY column` clause. For example, order states alphabetically.

In [ ]:
%%sql

SELECT * FROM states ORDER BY state_name;

To order the results in descending order, use the `ORDER BY column DESC` clause. For example, rank countries by population in 2023.

In [ ]:
%%sql

SELECT country_name, year, value AS population
FROM population
WHERE year = 2023
ORDER BY value DESC;

## The WHERE Clause

The `WHERE` clause is used to filter records. It extracts only those records that fulfill a specified condition.

In [ ]:
%%sql

SELECT * FROM population WHERE country_name = 'Sudan' AND year >= 2020;

You can use boolean operators such as `AND`, `OR`, `NOT` to filter records.

In [ ]:
%%sql

SELECT country_name, year, value
FROM population
WHERE (country_name = 'Sudan' OR country_name = 'Egypt')
AND year >= 2020
ORDER BY country_name, year;

To select states with names starting with the letter `N`, use `LIKE 'N%'`.

In [ ]:
%%sql

SELECT * FROM states WHERE state_name LIKE 'N%';

To select all Darfur states, use `LIKE '%Darfur%'`.

In [ ]:
%%sql

SELECT * FROM states WHERE state_name LIKE '%Darfur%';

To select all Kordofan states, use `LIKE '%Kordofan%'`.

In [ ]:
%%sql

SELECT * FROM states WHERE state_name LIKE '%Kordofan%';

To select from a list of values, use the `IN` operator.

In [ ]:
%%sql

SELECT country_name, year, value
FROM population
WHERE country_name IN ('Sudan', 'Egypt', 'Ethiopia')
AND year = 2023
ORDER BY value DESC;

To select rows within a range, use the `BETWEEN` operator. For example, population data between 2015 and 2023.

In [ ]:
%%sql

SELECT country_name, year, value
FROM population
WHERE country_name = 'Sudan'
AND year BETWEEN 2015 AND 2023
ORDER BY year;

## SQL Joins

Reference: https://www.w3schools.com/sql/sql_join.asp

Here are the different types of JOINs in SQL:

- `(INNER) JOIN`: Returns records that have matching values in both tables
- `LEFT (OUTER) JOIN`: Returns all records from the left table, and the matched records from the right table
- `RIGHT (OUTER) JOIN`: Returns all records from the right table, and the matched records from the left table
- `FULL (OUTER) JOIN`: Returns all records when there is a match in either left or right table

![](https://i.imgur.com/mITYzuS.png)

We have two sample tables: `population` and `gdp`.

Both contain data for 8 countries across many years. We'll join them to calculate **GDP per capita**.

In [ ]:
%%sql

SELECT COUNT(*) AS population_rows FROM population;

In [ ]:
%%sql

SELECT COUNT(*) AS gdp_rows FROM gdp;

In [ ]:
%%sql

SELECT * FROM population WHERE year = 2023 ORDER BY country_name;

In [ ]:
%%sql

SELECT * FROM gdp WHERE year = 2023 ORDER BY country_name;

### SQL Inner Join

The `INNER JOIN` keyword selects records that have matching values in both tables. Here we join `population` and `gdp` on `country` and `year` to calculate GDP per capita.

In [ ]:
%%sql

SELECT
    p.country_name,
    p.year,
    p.value AS population,
    g.value AS gdp_usd,
    ROUND(g.value / p.value, 2) AS gdp_per_capita
FROM population p
INNER JOIN gdp g ON p.country = g.country AND p.year = g.year
WHERE p.year = 2023
ORDER BY gdp_per_capita DESC;

### SQL Left Join

The `LEFT JOIN` keyword returns all records from the left table (`population`), and the matched records from the right table (`gdp`). If there is no match, the right side will contain `NULL`.

In [ ]:
%%sql

SELECT
    p.country_name,
    p.year,
    p.value AS population,
    g.value AS gdp_usd
FROM population p
LEFT JOIN gdp g ON p.country = g.country AND p.year = g.year
WHERE p.year >= 2022
ORDER BY p.country_name, p.year;

### SQL Right Join

The `RIGHT JOIN` keyword returns all records from the right table (`gdp`), and the matched records from the left table (`population`).

In [ ]:
%%sql

SELECT
    g.country_name,
    g.year,
    p.value AS population,
    g.value AS gdp_usd
FROM population p
RIGHT JOIN gdp g ON p.country = g.country AND p.year = g.year
WHERE g.year >= 2022
ORDER BY g.country_name, g.year;

### SQL Full Join

The `FULL JOIN` keyword returns all records when there is a match in either left or right table.

In [ ]:
%%sql

SELECT
    COALESCE(p.country_name, g.country_name) AS country_name,
    COALESCE(p.year, g.year) AS year,
    p.value AS population,
    g.value AS gdp_usd
FROM population p
FULL JOIN gdp g ON p.country = g.country AND p.year = g.year
WHERE COALESCE(p.year, g.year) = 2023
ORDER BY country_name;

### SQL Union

The `UNION` operator is used to combine the result-set of two or more `SELECT` statements.

In [ ]:
%%sql

SELECT country_name, 'population' AS indicator FROM population WHERE year = 2023
UNION
SELECT country_name, 'gdp' AS indicator FROM gdp WHERE year = 2023
ORDER BY country_name, indicator;

## Aggregation

### Group By

The `GROUP BY` statement groups rows that have the same values into summary rows, like "find the latest population of each country".

The `GROUP BY` statement is often used with aggregate functions (`COUNT`, `MAX`, `MIN`, `SUM`, `AVG`) to group the result-set by one or more columns.

In [ ]:
%%sql

SELECT country_name, COUNT(*) AS num_years
FROM population
GROUP BY country_name
ORDER BY num_years DESC;

In [ ]:
%%sql

SELECT country_name, MAX(value) AS max_population, MIN(value) AS min_population
FROM population
GROUP BY country_name
ORDER BY max_population DESC;

Calculate the average population per decade for Sudan.

In [ ]:
%%sql

SELECT
    (year / 10) * 10 AS decade,
    ROUND(AVG(value), 0) AS avg_population
FROM population
WHERE country_name = 'Sudan'
GROUP BY decade
ORDER BY decade;

### Having

The `HAVING` clause was added to SQL because the `WHERE` keyword could not be used with aggregate functions.

For example, to select countries with a maximum population greater than 50 million:

In [ ]:
%%sql

SELECT country_name, MAX(value) AS max_population
FROM population
GROUP BY country_name
HAVING MAX(value) > 50000000
ORDER BY max_population DESC;

GDP per capita by country, only showing countries with GDP per capita > $1000.

In [ ]:
%%sql

SELECT
    p.country_name,
    ROUND(AVG(g.value / p.value), 2) AS avg_gdp_per_capita
FROM population p
INNER JOIN gdp g ON p.country = g.country AND p.year = g.year
WHERE p.year >= 2015
GROUP BY p.country_name
HAVING AVG(g.value / p.value) > 1000
ORDER BY avg_gdp_per_capita DESC;

## Conditional Statements

The `CASE` statement goes through conditions and returns a value when the first condition is met (like an `IF-THEN-ELSE` statement).

For example, to classify countries by population size:

In [ ]:
%%sql

SELECT country_name, value AS population,
CASE
    WHEN value > 100000000 THEN 'Very Large (100M+)'
    WHEN value > 30000000 THEN 'Large (30M+)'
    WHEN value > 10000000 THEN 'Medium (10M+)'
    ELSE 'Small (<10M)'
END AS size_category
FROM population
WHERE year = 2023
ORDER BY value DESC;

Classify Sudan's states into regions using `CASE`.

In [ ]:
%%sql

SELECT state_name, state_name_ar, iso_code,
CASE
    WHEN state_name LIKE '%Darfur%' THEN 'Darfur'
    WHEN state_name LIKE '%Kordofan%' THEN 'Kordofan'
    WHEN state_name IN ('Khartoum', 'Al Jazirah', 'White Nile', 'Blue Nile', 'Sennar') THEN 'Central'
    WHEN state_name IN ('Kassala', 'Al Qadarif', 'Red Sea') THEN 'Eastern'
    WHEN state_name IN ('River Nile', 'Northern') THEN 'Northern'
    ELSE 'Other'
END AS region
FROM states
ORDER BY region, state_name;

## Saving Results

You can save the results of a query to a new table using the `CREATE TABLE AS` statement.

In [ ]:
%%sql

DROP TABLE IF EXISTS sudan_summary;
CREATE TABLE sudan_summary AS
SELECT country_name, year, value AS population
FROM population
WHERE country_name = 'Sudan' AND year >= 2000;

In [ ]:
%%sql

FROM sudan_summary;

Use the `INSERT INTO` statement to insert rows into a table.

In [ ]:
%%sql

DROP TABLE IF EXISTS darfur_states;
CREATE TABLE darfur_states AS
SELECT * FROM states WHERE state_name LIKE '%Darfur%';

In [ ]:
%%sql

INSERT INTO darfur_states
SELECT * FROM states WHERE state_name LIKE '%Kordofan%';

In [ ]:
%%sql

FROM darfur_states;

## Export to Files

Export data to CSV.

In [ ]:
%%sql

COPY (SELECT * FROM sudan_summary) TO 'sudan_population.csv' (HEADER, DELIMITER ',');

## Geospatial Queries (Bonus)

The Sudan extension embeds real MultiPolygon boundaries (GADM v4.1) for all 18 states. Combined with DuckDB's `spatial` extension, you can run spatial SQL queries.

Install and load the spatial extension first.

In [ ]:
%%sql

RESET custom_extension_repository;
INSTALL spatial;
LOAD spatial;

Convert GeoJSON strings to geometry objects.

In [ ]:
%%sql

SELECT state_name, ST_GeomFromGeoJSON(geojson) AS geom
FROM SUDAN_Boundaries('state');

Compute centroids from polygon geometries.

In [ ]:
%%sql

SELECT
    state_name,
    ROUND(ST_X(ST_Centroid(ST_GeomFromGeoJSON(geojson))), 3) AS centroid_lon,
    ROUND(ST_Y(ST_Centroid(ST_GeomFromGeoJSON(geojson))), 3) AS centroid_lat
FROM SUDAN_Boundaries('state');

Find which state contains a specific point (e.g., Khartoum city at 32.53, 15.59).

In [ ]:
%%sql

SELECT state_name, state_name_ar
FROM SUDAN_Boundaries('state')
WHERE ST_Contains(ST_GeomFromGeoJSON(geojson), ST_Point(32.53, 15.59));

Compute approximate area of each state in km².

In [ ]:
%%sql

SELECT
    state_name,
    ROUND(ST_Area(ST_GeomFromGeoJSON(geojson)) * 111.32 * 111.32, 0) AS area_km2
FROM SUDAN_Boundaries('state')
ORDER BY area_km2 DESC;

Export boundaries to GeoPackage file (can be opened in QGIS, ArcGIS, etc.).

In [ ]:
%%sql

COPY (
    SELECT state_name, state_name_ar, iso_code,
           ST_GeomFromGeoJSON(geojson) AS geom
    FROM SUDAN_Boundaries('state')
) TO 'sudan_states.gpkg' WITH (FORMAT GDAL, DRIVER 'GPKG');

## SQL Comments

Comments are used to explain sections of SQL statements, or to prevent execution of SQL statements.

### Single Line Comments

Single line comments start with `--`. Any text between `--` and the end of the line will be ignored.

In [ ]:
%%sql

SELECT * FROM states LIMIT 5 -- Show first 5 states;

### Multi-line Comments

Multi-line comments start with `/*` and end with `*/`. Any text between `/*` and `*/` will be ignored.

In [ ]:
%%sql

SELECT country_name, year, value
FROM population
/*
 * Filter for Sudan only
 * Recent years (2020+)
 */
WHERE country_name = 'Sudan'
AND year >= 2020
ORDER BY year DESC;